In [ ]:
__author__ = 'Adriano Pieres <adriano.pieres@linea.gov.br, LIneA Team'
__version__ = '20190829' # yyyymmdd
__datasets__ = ''
__keywords__ = ['Jupyter','Python','QA','tutorial']

# 15 - Creating JN for Quality Assurance

## LIneA Bootcamp

**Contact person:** Adriano Pieres [adriano.pieres@linea.gov.br]

**Last verified run:** 28-Aug-2019


This notebook aims to show the properties of a small region of the DESDR1 catalog.
The star/galaxy separator is shown, determine completeness regarding any other survey,
as DES data to HSC, 2MASS, PanSTARRS and Gaia.

See more information about `DES-DR1 data` and its related papers on:

https://www.darkenergysurvey.org/dr1-data-release-papers/

Firstly, the imports and setups:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import healpy as hp
import os
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from matplotlib.colors import LogNorm
from matplotlib import rcParams
# Set uniform colormaps: magma, viridis, plasma, inferno
cmap = matplotlib.cm.get_cmap("inferno")
cmap.set_under('darkgray')
cmap.set_bad('darkgray')
matplotlib.rc('xtick', labelsize=8) 
matplotlib.rc('ytick', labelsize=8) 

Reading a small region of the sky:

In [ ]:
filename = '/archive/user/adriano.pieres/bootcamp_2019/DES_DR1_g24.fits'  
hdulist = fits.open(filename)
catalog = hdulist[1].data
print(catalog.columns)

Basically the catalog contains the position (ra, dec), _griz_ magnitudes and errors, and the star-galaxy classifier spread_model (sm) in three bands. All the sources are limited to _g=24._ 
<br>
The magnitudes listed are the MAG_PSF magnitudes, or the aperture flux fit with a Point Spread Function.
See more details in the SExtractor Manual:
<br>
http://astroa.physics.metu.edu.tr/MANUALS/sextractor/Guide2source_extractor.pdf
<br>
Let's read the columns of the catalog:

In [ ]:
RA = catalog.field('ra')
DEC = catalog.field('dec')

MAG_G = catalog.field('mag_g')
MAG_R = catalog.field('mag_r')
MAG_I = catalog.field('mag_i')
MAG_Z = catalog.field('mag_z')
MAGERR_G = catalog.field('magerr_g')
MAGERR_R = catalog.field('magerr_r')
MAGERR_I = catalog.field('magerr_i')
MAGERR_Z = catalog.field('magerr_z')

SM_G = catalog.field('smg')
SM_R = catalog.field('smr')
SM_I = catalog.field('smi')
SMERR_G = catalog.field('smgerr')
SMERR_R = catalog.field('smrerr')
SMERR_I = catalog.field('smierr')

hdulist.close()

There is something interesting in **astropy.fits.open**. Usually, you open the file, read the columns,
and close the file. But when you do that, the file is opened, the columns are not loaded in memory and
the file is not closed. That is a trick to _load_ columns very fast. The columns are only read when you
apply them, as when you count the elements in _ra_. It may cause problems when you open many files...
<br>
Now, printing the total number of objects and its limits in RA and DEC:

In [ ]:
print('Total number of objects: ', len(RA))
print('RA limits: ', RA.min(), RA.max())
print('DEC limits: ', DEC.min(), DEC.max())

Let's plot the position of the patch regarding the footprint of the DES. Loadiing the limits of the survey:

In [ ]:
RA_DES, DEC_DES = np.loadtxt('/archive/user/adriano.pieres/des-round17-poly.txt', usecols=(0, 1), unpack=True)

and now printing the region and the footprint map:

In [ ]:
# Equatorial Coordinates:
plt.scatter(360.-RA, DEC, s=0.01, color='b')
plt.grid(True)
plt.xlim(RA_DES.max()+3, RA_DES.min()-3)
plt.ylim(DEC_DES.min()-3, DEC_DES.max()+3)
plt.plot(RA_DES, DEC_DES, 'k')
plt.xlabel(r'$\alpha$(deg)')
plt.ylabel(r'$\delta$(deg)')

An equatorial region sampled in Stripe 82.
**Star-galaxy classifier**
Spread_model is the difference (in magnitudes) between the fit of a PSF and the fit of a
PSF convolved with an exponential model. It is a good star-galaxy classifier when the shape
of the source is discernible.

To evaluate the performance of the spread-model, we plot two color-color diagrams with the 
help of the CCD function.

In [ ]:
def CC(RA_, DEC_, mag1, mag2, mag3, mag4, maglim, mode):
    '''
    This function is designed for DES data.
    - Inputs:
    RA: longitude in Equatorial coordinates (array, units: degrees)
    DEC: latitude in Equatorial coordinates (array, units: degrees)
    magN: magnitude in N band (array)
    maglim: limiting magnitude in the first band (g)
    mode: 'linear' or 'log' (string)
    - Output:
    Figure with 2 2D histograms showing the color-color diagrams (g-r x r-i and r-i x i-z)
    '''
    # First, transform the Eq to Gal coordinates in order to calculate the extinction
    c = SkyCoord(ra=RA_ * u.degree, dec=DEC_ * u.degree)
    L, B = c.galactic.l.degree, c.galactic.b.degree

    bright = (mag1 <= maglim)
    L = L[bright]
    B = B[bright]
    mag1 = mag1[bright]
    mag2 = mag2[bright]
    mag3 = mag3[bright]
    mag4 = mag4[bright]

    # remove the extinction in respective band
    ngp = fits.open('/archive/external_catalogs/SDSS3/schlegel/schlegel/SFD_dust_4096_ngp.fits')[0].data
    sgp = fits.open('/archive/external_catalogs/SDSS3/schlegel/schlegel/SFD_dust_4096_sgp.fits')[0].data

    n = np.array(B > 0).astype(int)
    n[n == 0] = -1

    x = 2048 * n * (1 - (n * np.sin(np.deg2rad(B))) ** 0.5) * np.cos(np.deg2rad(L)) + 2047.5
    y = -2048 * n * (1 - (n * np.sin(np.deg2rad(B))) ** 0.5) * np.sin(np.deg2rad(L)) + 2047.5
    x, y = np.round(x, 0).astype(int), np.round(y, 0).astype(int)

    EBV_SFD98 = np.zeros(len(B))
    EBV_SFD98[B > 0] = ngp[y[B > 0], x[B > 0]]
    EBV_SFD98[B < 0] = sgp[y[B < 0], x[B < 0]]

    AV = 3.1 * EBV_SFD98
    AG = 1.17255 * AV
    AR = 0.83414 * AV
    AI = 0.63700 * AV
    AZ = 0.47025 * AV
    
    mag1 -= AG
    mag2 -= AR
    mag3 -= AI
    mag4 -= AZ

    f, ((ax1, ax2)) = plt.subplots(1, 2, figsize=(8, 8), dpi=150)
    x = mag1-mag2
    y = mag2-mag3
    z = mag3-mag4

    H, xedges, yedges = np.histogram2d(x, y, bins=200, range=[[-1, 2.2],[-1,2.2]])
    ax1.set_title(r'CCD ($g$ - $r$ x $r$ - $i$)', fontsize=10)
    ax1.set_xlim([-1, 2.2])
    ax1.set_ylim([-1, 2.2])
    ax1.set_xlabel(r'$g$ - $r$', fontsize=10)
    ax1.set_ylabel(r'$r$ - $i$', fontsize=10)
    ax1.grid(True, lw=0.2)
    if (mode=='log'):
        H[H == 0] = 0.1
        im1 = ax1.imshow(np.flipud(H.T), extent=[-1, 2.2, -1, 2.2], aspect='equal', interpolation='None', cmap=cmap, norm=LogNorm(vmin=np.min(H), vmax=np.max(H)))
    else:
        im1 = ax1.imshow(np.flipud(H.T), extent=[-1, 2.2, -1, 2.2], aspect='equal', interpolation='None', cmap=cmap, vmin=np.min(H), vmax=np.max(H))

    G, xedges, yedges = np.histogram2d(y, z, bins=200, range=[[-1, 2.2],[-1,2.2]])
    ax2.set_title(r'CCD ($r$ - $i$ x $i$ - $z$)', fontsize=10)
    ax2.set_xlim([-1, 2.2])
    ax2.set_ylim([-1, 2.2])
    ax2.set_xlabel(r'$r$ - $i$', fontsize=10)
    ax2.set_ylabel(r'$i$ - $z$', fontsize=10)
    ax2.grid(True, lw=0.2)
    if (mode=='log'):
        G[G == 0] = 0.1
        im2 = ax2.imshow(np.flipud(G.T), extent=[-1, 2.2, -1, 2.2], aspect='equal', interpolation='None', cmap=cmap, norm=LogNorm(vmin=np.min(H), vmax=np.max(H)))
    else:
        im2 = ax2.imshow(np.flipud(G.T), extent=[-1, 2.2, -1, 2.2], aspect='equal', interpolation='None', cmap=cmap, vmin=np.min(H), vmax=np.max(H))

    cbaxes = f.add_axes([0.98, 0.3, 0.015, 0.398])
    cbar = f.colorbar(im1, cax=cbaxes, cmap=cmap, orientation='vertical')
    cbar.ax.set_xticklabels(np.linspace(0., np.max(G), 5),rotation=0, fontsize=8)
    plt.tight_layout()
    del(RA_, DEC_, L, B, mag1, mag2, mag3, mag4, mode, maglim)

In [ ]:
CC(RA, DEC, MAG_G, MAG_R, MAG_I, MAG_Z, 24., 'log')

Clearly the stellar locus and the galaxy locus are overlapping on the color-color diagram. Let's apply the spread-model as SG classifier, defining:

In [ ]:
star = (np.abs(SM_I) <= 0.003)
galaxy = ~star

The Color-color diagram for stars is:

In [ ]:
CC(RA[star], DEC[star], MAG_G[star], MAG_R[star], MAG_I[star], MAG_Z[star], 24., 'log')

and for galaxies:

In [ ]:
CC(RA[galaxy], DEC[galaxy], MAG_G[galaxy], MAG_R[galaxy], MAG_I[galaxy], MAG_Z[galaxy], 24., 'log')

In [ ]:
f, ((ax1, ax2, ax3)) = plt.subplots(1, 3, figsize=(8, 2), sharey=True, dpi=150)

ax1.hist2d(MAG_G, SM_I, bins=[200,200], range=[(14,24),(-0.01,0.025)], cmap=cmap, norm=LogNorm())
ax1.set_xlabel('MAG_AUTO_G')
ax1.set_ylabel('spread_model')

ax2.hist2d(MAG_R, SM_R, bins=[200,200], range=[(14,24),(-0.01,0.025)], cmap=cmap, norm=LogNorm())
ax2.set_xlabel('MAG_AUTO_R')

ax3.hist2d(MAG_I, SM_I, bins=[200,200], range=[(14,24),(-0.01,0.025)], cmap=cmap, norm=LogNorm())
ax3.set_xlabel('MAG_AUTO_I')

plt.subplots_adjust(wspace=0, hspace=0)

It seems that the best work is done by the SM in the ** g ** band instead that in the ** i ** band.
Let's plot using as reference the SM in the g band:

In [ ]:
star_g = (np.abs(SM_G) <= 0.002)
galaxy_g = ~star_g

CC(RA[star_g], DEC[star_g], MAG_G[star_g], MAG_R[star_g], MAG_I[star_g], MAG_Z[star_g], 23., 'log')

### Completeness ###

In order to check the completeness of the catalog, we compare the DES data to deeper
data from Hyper Suprime Camera, installed on Subaru (8m). The fields DEEP2 and DEEP3 were
previously stored in the devel environment:

In [ ]:
file_HSC = '/archive/user/adriano.pieres/HSC_data/HSC_DEEP23_VVDS_UDEEP.fits'  
hduHSC = fits.open(file_HSC)
catalog_HSC = hduHSC[1].data

RA_HSC = catalog_HSC.field('ra')
DEC_HSC = catalog_HSC.field('dec')
MAGG_PSF_HSC = catalog_HSC.field('gmag_psf')
MAGR_PSF_HSC = catalog_HSC.field('rmag_psf')
MAGI_PSF_HSC = catalog_HSC.field('imag_psf')
MAGZ_PSF_HSC = catalog_HSC.field('zmag_psf')
ICMODEL_HSC = catalog_HSC.field('icmodel_mag')
hduHSC.close()

The DEEP2 and 3 uses the ICMODEL, that is similar to spread_model as SG classifier. Let's take a look on the 
distribution of data regarding magnitude and ICMODEL:

In [ ]:
plt.hist2d(MAGI_PSF_HSC, MAGI_PSF_HSC-ICMODEL_HSC, bins=[200,200], range=[(18.,31.),(-.03,1.)], cmap=cmap, norm=LogNorm())
plt.xlabel('MAGG_PSF_HSC')
plt.ylabel('ICMODEL_HSC')
plt.plot(np.linspace(18.,30.,10), np.repeat(0.03,10), 'b:')

In [ ]:
STAR_HSC = MAGI_PSF_HSC - ICMODEL_HSC < 0.03
GALAXY_HSC = MAGI_PSF_HSC - ICMODEL_HSC >= 0.03

c_HSC = SkyCoord(ra=RA_HSC * u.degree, dec=DEC_HSC * u.degree)
L_HSC, B_HSC = c_HSC.galactic.l.degree, c_HSC.galactic.b.degree

To compare the DESDR1 to HSC data, I created a completeness function that works as follows.
The footprint of both survey are mapped, and an intersection between then is defined. Based
on the intersection map, the object's counts are compared.

In [ ]:
def Compl(data1, data2, magmin, magmax, binmag, title, nside=512):
    '''
    - Input data:
    data1 = np.stack((RA,
                      DEC,
                      MAGG,
                      MAGR,
                      MAGG,
                      MAGR), axis=0)
    data2 has similar format, but for second survey, obviously
    magmin, magmax, binmag: limiting magnitudes and magnitude bin(floats)
    title: (string)
    nside: HealPix nside (2^N, from 2^0 to 2^30). Nside = 512 produces a pixel with side = 7 arcmin.
    Click on the blue bar on the left to collapse this cell. (Amazing!)
    '''
    # Footprint map of the first survey
    HPX1 = hp.ang2pix(nside, np.pi / 2. - np.radians(data1[1][:]), np.radians(data1[0][:]), nest=True)
    # Same for the second survey
    HPX2 = hp.ang2pix(nside, np.pi / 2. - np.radians(data2[1][:]), np.radians(data2[0][:]), nest=True)

    # Creating an array with unique pixels
    HPX1_un = np.unique(HPX1)
    HPX2_un = np.unique(HPX2)

    # Determining the intersection between both maps
    HPX_intersec = np.intersect1d(HPX1_un, HPX2_un)

    # Creating an identifier to each survey
    idx1 = [i for i, e in enumerate(HPX1) if e in HPX_intersec]
    idx2 = [i for i, e in enumerate(HPX2) if e in HPX_intersec]
    
    # Histogram of each magnitude (griz) for both surveys
    g1_n, g1_xbins, g1_ybins = plt.hist(data1[2][idx1], bins=np.arange(magmin,magmax,binmag), log=False)
    r1_n, r1_xbins, r1_ybins = plt.hist(data1[3][idx1], bins=np.arange(magmin,magmax,binmag), log=False)
    i1_n, i1_xbins, i1_ybins = plt.hist(data1[4][idx1], bins=np.arange(magmin,magmax,binmag), log=False)
    z1_n, z1_xbins, z1_ybins = plt.hist(data1[5][idx1], bins=np.arange(magmin,magmax,binmag), log=False)

    g2_n, g2_xbins, g2_ybins = plt.hist(data2[2][idx2], bins=np.arange(magmin,magmax,binmag), log=False)
    r2_n, r2_xbins, r2_ybins = plt.hist(data2[3][idx2], bins=np.arange(magmin,magmax,binmag), log=False)
    i2_n, i2_xbins, i2_ybins = plt.hist(data2[4][idx2], bins=np.arange(magmin,magmax,binmag), log=False)
    z2_n, z2_xbins, z2_ybins = plt.hist(data2[5][idx2], bins=np.arange(magmin,magmax,binmag), log=False)
    
    # The errors implied in the histograms
    g1_less = g1_n - np.sqrt(g1_n)
    g1_more = g1_n + np.sqrt(g1_n)
    r1_less = r1_n - np.sqrt(r1_n)
    r1_more = r1_n + np.sqrt(r1_n)
    i1_less = i1_n - np.sqrt(i1_n)
    i1_more = i1_n + np.sqrt(i1_n)
    z1_less = z1_n - np.sqrt(z1_n)
    z1_more = z1_n + np.sqrt(z1_n)

    plt.clf()
    
    comp_g = g1_n/g2_n; comp_r = r1_n/r2_n; comp_i = i1_n/i2_n; comp_z = z1_n/z2_n
    comp_g_less = g1_less/g2_n
    comp_r_less = r1_less/r2_n
    comp_i_less = i1_less/i2_n
    comp_z_less = z1_less/z2_n
    comp_g_more = g1_more/g2_n
    comp_r_more = r1_more/r2_n
    comp_i_more = i1_more/i2_n
    comp_z_more = z1_more/z2_n

    # Feel free to force completeness to 1 regarding the second survey
    #comp_g[comp_g >1.] = 1.;
    comp_g[(g1_n == 0.)] = 0
    #comp_r[comp_r >1.] = 1.;
    comp_r[(r1_n == 0.)] = 0
    #comp_i[comp_i >1.] = 1.;
    comp_i[(i1_n == 0.)] = 0
    #comp_z[comp_z >1.] = 1.;
    comp_z[(z1_n == 0.)] = 0

    #comp_g_less[comp_g_less > 1.] = 1.;
    comp_g_less[(g1_n == 0.)|(g2_n == 0.)] = 0
    #comp_r_less[comp_r_less > 1.] = 1.;
    comp_r_less[(r1_n == 0.)|(r2_n == 0.)] = 0
    #comp_i_less[comp_i_less > 1.] = 1.;
    comp_i_less[(i1_n == 0.)|(i2_n == 0.)] = 0
    #comp_z_less[comp_z_less > 1.] = 1.;
    comp_z_less[(z1_n == 0.)|(z2_n == 0.)] = 0

    #comp_g_more[comp_g_more > 1.] = 1.;
    comp_g_more[(g1_n == 0.)|(g2_n == 0.)] = 0
    #comp_r_more[comp_r_more > 1.] = 1.;
    comp_r_more[(r1_n == 0.)|(r2_n == 0.)] = 0
    #comp_i_more[comp_i_more > 1.] = 1.;
    comp_i_more[(i1_n == 0.)|(i2_n == 0.)] = 0
    #comp_z_more[comp_z_more > 1.] = 1.;
    comp_z_more[(z1_n == 0.)|(z2_n == 0.)] = 0
    
    # Plotting completenesses in same figure
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=True, sharey=True)
    ax1.plot(g1_xbins[:-1], comp_g, color='g', lw=1, label='g')
    ax1.set_ylabel('ratio')
    ax3.set_ylabel('ratio')
    ax3.set_xlabel('magnitude')
    ax4.set_xlabel('magnitude')
    ax2.plot(g1_xbins[:-1], comp_r, color='r', lw=1, label='r')
    ax3.plot(g1_xbins[:-1], comp_i, color='maroon', lw=1, label='i')
    ax4.plot(g1_xbins[:-1], comp_z, color='k', lw=1, label='z')
    ax1.legend()
    ax2.legend()
    ax3.legend()
    ax4.legend()
    ax1.grid(True, lw=0.2)
    ax2.grid(True, lw=0.2)
    ax3.grid(True, lw=0.2)
    ax4.grid(True, lw=0.2)
    ax1.fill_between(g1_xbins[:-1], comp_g_more, comp_g_less, facecolor='aquamarine')
    ax2.fill_between(g1_xbins[:-1], comp_r_more, comp_r_less, facecolor='bisque')
    ax3.fill_between(g1_xbins[:-1], comp_i_more, comp_i_less, facecolor='lavender')
    ax4.fill_between(g1_xbins[:-1], comp_z_more, comp_z_less, facecolor='honeydew')
    ax1.set_ylim([0.,3.])
    ax1.set_xlim([magmin, magmax])
    plt.legend()
    plt.suptitle(title)
    plt.grid(True)
    plt.subplots_adjust(hspace=0.0, wspace=0.0)

Applying this function to DES and HSC data:

In [ ]:
data_DES_star = np.stack((RA[star],
                         DEC[star],
                         MAG_G[star],
                         MAG_R[star],
                         MAG_I[star],
                         MAG_Z[star]), axis=0)
data_HSC_star = np.stack((RA_HSC[(STAR_HSC)],
                         DEC_HSC[(STAR_HSC)],
                         MAGG_PSF_HSC[(STAR_HSC)],
                         MAGR_PSF_HSC[(STAR_HSC)],
                         MAGI_PSF_HSC[(STAR_HSC)],
                         MAGZ_PSF_HSC[(STAR_HSC)]), axis=0)

In [ ]:
Compl(data_DES_star, data_HSC_star, 19., 26.2, 0.2, 'Star (complete) ratio DESDR1/HSC', 1024)

In [ ]:
data_DES_galaxy = np.stack((RA[galaxy],
                         DEC[galaxy],
                         MAG_G[galaxy],
                         MAG_R[galaxy],
                         MAG_I[galaxy],
                         MAG_Z[galaxy]), axis=0)
data_HSC_galaxy = np.stack((RA_HSC[(GALAXY_HSC)],
                         DEC_HSC[(GALAXY_HSC)],
                         MAGG_PSF_HSC[(GALAXY_HSC)],
                         MAGR_PSF_HSC[(GALAXY_HSC)],
                         MAGI_PSF_HSC[(GALAXY_HSC)],
                         MAGZ_PSF_HSC[(GALAXY_HSC)]), axis=0)

In [ ]:
Compl(data_DES_galaxy, data_HSC_galaxy, 19., 26.2, 0.2, 'Galaxy (complete) ratio DESDR1/HSC', 1024)

### PanStaRRS
<br>
An astrometric comparison to PanStaRRs can be done here. Files with PanStaRRs stars (sorry!) are stored
in the LIneA's machine. The files are stored in HealPix pixels, with nside=32 and nest=True. Follow
the link below to find the files.

In [ ]:
HPX32 = hp.ang2pix(32, np.pi / 2. - np.radians(DEC), np.radians(RA), nest=True)
HPX32_un = np.unique(HPX32)

RA_PS_, DEC_PS_, GFPSFMAG_PS_, RFPSFMAG_PS_, IFPSFMAG_PS_, GFPSFMAGERR_PS_, \
RFPSFMAGERR_PS_, IFPSFMAGERR_PS_ = [], [], [], [], [], [], [], []

for i in range(len(HPX32_un)):
    try:
        PS_file = '/archive/external_catalogs/PANSTARRS/healpix/32/'+str(np.int(HPX32_un[i]))+'.fits'
        hdu = fits.open(PS_file)
        PS_cat = hdu[1].data
        RA_PS = PS_cat.field('RA')
        DEC_PS = PS_cat.field('DEC')
        GFPSFMAG_PS = PS_cat.field('GFPSFMAG')
        RFPSFMAG_PS = PS_cat.field('RFPSFMAG')
        IFPSFMAG_PS = PS_cat.field('IFPSFMAG')
        GFPSFMAGERR_PS = PS_cat.field('GFPSFMAGERR')
        RFPSFMAGERR_PS = PS_cat.field('RFPSFMAGERR')
        IFPSFMAGERR_PS = PS_cat.field('IFPSFMAGERR')
        hdu.close()
        RA_PS_.append(RA_PS)
        DEC_PS_.append(DEC_PS)
        GFPSFMAG_PS_.append(GFPSFMAG_PS)
        RFPSFMAG_PS_.append(RFPSFMAG_PS)
        IFPSFMAG_PS_.append(IFPSFMAG_PS)
        GFPSFMAGERR_PS_.append(GFPSFMAGERR_PS)
        RFPSFMAGERR_PS_.append(RFPSFMAGERR_PS)
        IFPSFMAGERR_PS_.append(IFPSFMAGERR_PS)
        print('This HPixel '+str(HPX32_un[i])+' was sampled by Panstarrs.')
    except:
        print('This HPixel '+str(HPX32_un[i])+' was not sampled by Panstarrs.')

And since you upload these quantities in the system, you are able to compare both surveys,
in the same way as we have done before.

### 2MASS ###
<br>
* Astrometric comparison *
<br>
Firstly, downloading the data. From IRSA the single option is a Cone Radius (see https://irsa.ipac.caltech.edu/docs/howto/gator_prog_interface.html).
Let's estimate this Cone Radius by the range in declination, minding the maximum is one degree:

In [ ]:
cone_radius = min(0.5 * (DEC.max() - DEC.min()), 1.)

Now, downloading data from IRSA/IPAC/CALTECH. This command has a limit for the ConeRadius not informed in the help page. Also, bear in mind that this command would be done only in the first time. After that, comment this line.

In [ ]:
os.system('wget "http://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-query?outfmt=1&objstr="'\
          +str(round(np.mean(RA),5))+'+'+str(round(np.mean(DEC),5))\
          +'"&spatial=Cone&radius='+str(cone_radius*3600)+'&catalog=fp_psc" -O CAT_2MASS.tbl')

In [ ]:
RA_TWOMASS, DEC_TWOMASS = np.loadtxt('CAT_2MASS.tbl', skiprows=103, usecols=(0,1,), dtype=float, unpack=True)

If you are interested in match those stars, you this astropy tool is very interesting to you:

In [ ]:
DES_CAT = SkyCoord(ra=RA*u.degree, dec=DEC*u.degree)
TWOMASS_CAT = SkyCoord(ra=RA_TWOMASS*u.degree, dec=DEC_TWOMASS*u.degree)

id_TWOMASS_DES, d2d_DES, d3d = DES_CAT.match_to_catalog_sky(TWOMASS_CAT)
id_DES_TWOMASS, d2d_TWOMASS, d3d = TWOMASS_CAT.match_to_catalog_sky(DES_CAT)

# Converting d2d distances from deg into arcsecs for sources closer than 1 arcsec:
sep_arcsec = d2d_DES.arcsecond[d2d_DES.arcsecond <= 1.0]

In [ ]:
a = plt.hist(sep_arcsec, bins=np.arange(0,1,0.01),
             color='b', alpha = 0.75, lw=2., histtype='step', log=False,
             label=str("""Mean: {0:4.0f} mas""".format(1000*np.mean(sep_arcsec))))
plt.ylabel('N')
plt.title('2MASS x DESDR1 angular separation')
plt.ylim([1,1.1*np.max(a[0])])
plt.xlabel('separation (arcsec)')
plt.legend()